In [1]:
import os
import tensorflow as tf
from tensorflow.python.framework import graph_io

import model

## Настраиваем Параметры

In [2]:
OUTPUT_NAMES = ['net/fc8/BiasAdd'] # ADJUST
INPUT_SIZE = [1, 224, 224, 3] # ADJUST
MODEL_PATH = 'output/vggA_BN/' # ADJUST
FROZEN_FPATH = 'data/frozen.pb' # ADJUST

## Создаем графы для удобства

In [5]:
graph = tf.Graph()
with graph.as_default():    
    with tf.variable_scope('net'):
        net_inp = tf.placeholder(tf.float32, INPUT_SIZE, name='input')
        net_out = model.model(net_inp, is_training=False)
    saver = tf.train.Saver()

## Начинаем инициализацию

In [6]:
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True
sess = tf.Session(graph=graph, config=sess_config)
snapshot_fpath = tf.train.latest_checkpoint(MODEL_PATH)
saver.restore(sess, snapshot_fpath)

ValueError: Can't load save_path when it is None.

## "Морозим" графы

In [7]:
graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())

graphdef_frozen = tf.graph_util.convert_variables_to_constants(
    sess, graphdef_inf, OUTPUT_NAMES)

graph_io.write_graph(graphdef_frozen, './', FROZEN_FPATH, as_text=False)

FailedPreconditionError: Attempting to use uninitialized value net/conv5_2/weights
	 [[Node: _retval_net/conv5_2/weights_0_31 = _Retval[T=DT_FLOAT, index=31, _device="/job:localhost/replica:0/task:0/device:CPU:0"](net/conv5_2/weights)]]

## Смотрим что заморозили

In [8]:
[x.name for x in graphdef_frozen.node]

['net/input',
 'net/Sub/y',
 'net/Sub',
 'net/conv1_1/weights',
 'net/conv1_1/Conv2D',
 'net/conv1_1/BatchNorm/Const',
 'net/conv1_1/BatchNorm/beta',
 'net/conv1_1/BatchNorm/moving_mean',
 'net/conv1_1/BatchNorm/moving_variance',
 'net/conv1_1/BatchNorm/FusedBatchNorm',
 'net/conv1_1/Relu',
 'net/pool1/MaxPool',
 'net/conv2_1/weights',
 'net/conv2_1/Conv2D',
 'net/conv2_1/BatchNorm/Const',
 'net/conv2_1/BatchNorm/beta',
 'net/conv2_1/BatchNorm/moving_mean',
 'net/conv2_1/BatchNorm/moving_variance',
 'net/conv2_1/BatchNorm/FusedBatchNorm',
 'net/conv2_1/Relu',
 'net/pool2/MaxPool',
 'net/conv3_1/weights',
 'net/conv3_1/Conv2D',
 'net/conv3_1/BatchNorm/Const',
 'net/conv3_1/BatchNorm/beta',
 'net/conv3_1/BatchNorm/moving_mean',
 'net/conv3_1/BatchNorm/moving_variance',
 'net/conv3_1/BatchNorm/FusedBatchNorm',
 'net/conv3_1/Relu',
 'net/conv3_2/weights',
 'net/conv3_2/Conv2D',
 'net/conv3_2/BatchNorm/Const',
 'net/conv3_2/BatchNorm/beta',
 'net/conv3_2/BatchNorm/moving_mean',
 'net/conv3_

## Доступная визуализация

In [7]:
graph_frozen = tf.Graph()
with graph_frozen.as_default():
    tf.import_graph_def(graphdef_frozen)
_=tf.summary.FileWriter('output/vggA_BN_frozen/', graph_frozen)